# Generic Rule Induction Notebook

Continuoulsy refined.

## README

- BRCG runs with proper (conda) aix360 environment
- Use aix360i environment for RIPPER

### Configuration


In [5]:
#from config_copy import config_dict
from config_copy import config_dict_imbalanced
from config_copy import Config_list

# document config order
#CONFIG = config_dict_imbalanced["CONFIG-I2"]
#CONFIG = config_dict["CONFIG14"]

#print('Proceed with configuration:', CONFIG["NAME"])

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
# import os
from sklearn.model_selection import train_test_split #, GridSearchCV
from sklearn.metrics import matthews_corrcoef,fbeta_score,confusion_matrix,f1_score,precision_score, recall_score, accuracy_score, balanced_accuracy_score, confusion_matrix, r2_score, explained_variance_score, mean_absolute_error, max_error
from sklearn.ensemble import GradientBoostingRegressor
#import wittgenstein as lw
import time
import warnings
import re

from aix360.algorithms.rbm import BRCGExplainer, BooleanRuleCG

from aix360.algorithms.rbm import FeatureBinarizer

from aix360.algorithms.rbm import FeatureBinarizerFromTrees

from aix360i.algorithms.rule_induction.ripper import Ripper

from corels import *




### Data

In [7]:
from collections import defaultdict
for config in config_dict_imbalanced:
    print(config)

CONFIG-I1
CONFIG-I2
CONFIG-I3
CONFIG-I4
CONFIG-I5
CONFIG-I6
CONFIG-I7
CONFIG-I8


In [8]:

metric_dict = {}
metric_list = []  
for config in config_dict_imbalanced:
    CONFIG = config_dict_imbalanced[config]
    def convert(char):
        if char == CONFIG['POS_CLASS']:
            return 1
        else:
            return 0

    df = pd.read_csv(CONFIG['DATA_SET'],dtype=CONFIG['DATA_TYPES'], nrows=15000)
    df = df.drop(columns=CONFIG['DROP'])
    if CONFIG['ALGO'] == 'BRCG':
        df[CONFIG['TARGET_LABEL']] = df[CONFIG['TARGET_LABEL']].map(convert)
        CONFIG['POS_CLASS'] = 1
    # maybe this could also be achieved through explicit binarization of target vector
    df.info()
    df[CONFIG['TARGET_LABEL']].value_counts()

    # train test split

    if CONFIG['TRAIN_TEST_SPLIT'] == 'FIXED':
        if CONFIG['MODE'] == 'PREDICTIVE':
            train = df[df['is_test_set'] == False]
            test = df[df['is_test_set'] == True]
        elif CONFIG['MODE'] == 'DESCRIPTIVE':
            train = df
            test = df

        train = train.drop(columns=['is_test_set'])
        test = test.drop(columns=['is_test_set'])

        y_train = train[CONFIG['TARGET_LABEL']]
        x_train = train.drop(columns=[CONFIG['TARGET_LABEL']])

        y_test = test[CONFIG['TARGET_LABEL']]
        x_test = test.drop(columns=[CONFIG['TARGET_LABEL']])
    else:
        x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=[CONFIG['TARGET_LABEL']]), df[CONFIG['TARGET_LABEL']], test_size=CONFIG['TRAIN_TEST_SPLIT'], random_state=42)

    print('Training:', x_train.shape, y_train.shape)
    print('Test:', x_test.shape, y_test.shape)

    for i in CONFIG["BINARIZER"]:
        
        
        if i == "TREES":
            binarizer =  FeatureBinarizerFromTrees(negations=True, randomState=42) 
            binarizer = binarizer.fit(x_train, y_train)
            x_train_bin = binarizer.transform(x_train) 
            x_test_bin = binarizer.transform(x_test)
            for algo in CONFIG["ALGO"]:
                präds = []
                if algo == 'RIPPER':
 
                    # start time
                    start_time = time.time()
                    try:
                        estimator = Ripper()
                        estimator.fit(x_train_bin, y_train, pos_value=CONFIG['POS_CLASS'])
                        end_time = time.time()
                        y_pred = estimator.predict(x_test_bin)
                        print('------------------------------------------------------') 
                        print('RIPPER TREES')
                        #print('Accuracy:', accuracy_score(y_test, y_pred))
                        #print('Balanced accuracy:', balanced_accuracy_score(y_test, y_pred))
                        #print('Precision:', precision_score(y_test, y_pred))
                        #print('Recall:', recall_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
                        ripper_t_bacc = balanced_accuracy_score(y_test, y_pred, adjusted=True)
                        ripper_t_f2 = fbeta_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS'], beta= 2)
                        ripper_t_acc = accuracy_score(y_test, y_pred)
                        #print('F1', f1_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
                        #print('ConfusionMatrix', confusion_matrix(y_test, y_pred))
                        #print('F-2', fbeta_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS'], beta= 2))
                        #print('Mathhews', matthews_corrcoef(y_test, y_pred))
                        print('Rule count: ' + str(sum([len(rules) for rules in estimator.rule_map.values()])))
                        ripper_t_rl = str(sum([len(rules) for rules in estimator.rule_map.values()]))
                        if ripper_t_rl != 0:
                        
                            for i in range(len(estimator.rule_map[CONFIG['POS_CLASS']])):
                                präds.append(len(estimator.rule_map[CONFIG['POS_CLASS']][i]))
                            ripper_t_präd = sum(präds)
                            ripper_t_max_präd = max(präds)
                        
                        else:
                            ripper_t_präd = 0
                            ripper_t_max_präd= 0
                    #print("Sum Prädikate:", sum(präds))
                    #print("MAX Prädikate:", max(präds))
                    #prec_t_ripper.append(precision_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
                    #rec_t_ripper.append(recall_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
                    #rip_t_rl.append(str(sum([len(rules) for rules in estimator.rule_map.values()])))
                        
                        print('------------------------------------------------------')
                    except Exception:
                        pass
                if algo == 'BRCG':
                    start_time = time.time()
                    estimator = BooleanRuleCG()
                    with warnings.catch_warnings():
                        warnings.simplefilter("ignore")
                        
                        estimator.fit(x_train_bin, y_train)
                        end_time = time.time()
                        y_pred = estimator.predict(x_test_bin)
                    print('------------------------------------------------------') 
                    #print('BRCG TREES')
                    #print('Accuracy:', accuracy_score(y_test, y_pred))
                    #print('Balanced accuracy:', balanced_accuracy_score(y_test, y_pred))
                    #print('Precision:', precision_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
                    #print('Recall:', recall_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
                    brcg_t_bacc = balanced_accuracy_score(y_test, y_pred, adjusted=True)
                    brcg_t_f2 = fbeta_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS'], beta= 2)
                    brcg_t_acc = accuracy_score(y_test, y_pred)
                    #print('F1', f1_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
                    #print('ConfusionMatrix', confusion_matrix(y_test, y_pred))
                    #print('F-2', fbeta_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS'], beta= 2))
                    #print('Mathhews', matthews_corrcoef(y_test, y_pred))
                    model = estimator.explain()
                    #print('Number of rules:', len(model['rules']))
                    brcg_t_rl = len(model['rules']) 
                    print('------------------------------------------------------')

                if algo == 'CORELS':
                    start_time = time.time()
                    estimator = CorelsClassifier(n_iter=10000, 
                        max_card=2, # feautres per statement
                        c = 0.0001 # Higher values penalise longer rulelists
                        )
                    estimator.fit(x_train_bin, y_train , prediction_name = CONFIG["TARGET_LABEL"])
                    end_time = time.time()
                    y_pred = estimator.predict(x_test_bin)
                    print('------------------------------------------------------') 
                    print('CORELS TREES')
                    #print('Accuracy:', accuracy_score(y_test, y_pred))
                    #print('Balanced accuracy:', balanced_accuracy_score(y_test, y_pred))
                    #print('Precision:', precision_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
                    #print('Recall:', recall_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
                    corels_t_bacc = balanced_accuracy_score(y_test, y_pred, adjusted=True)
                    corels_t_f2 = fbeta_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS'], beta= 2)
                    corels_t_acc = accuracy_score(y_test, y_pred)
                    #print('F1', f1_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
                    #print('ConfusionMatrix', confusion_matrix(y_test, y_pred))
                    #print('F-2', fbeta_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS'], beta= 2))
                    #print('Mathhews', matthews_corrcoef(y_test, y_pred))
                    r_length = len(estimator.rl().rules)
                    corels_t_rl = len(estimator.rl().rules)
                    print("Rule Length:", r_length)
                    print('------------------------------------------------------')

        if i == "QUANTILE":
            binarizer =  FeatureBinarizer(numThresh=9,negations=True, randomState=42) 
            binarizer = binarizer.fit(x_train)
            x_train_bin = binarizer.transform(x_train) 
            x_test_bin = binarizer.transform(x_test)  

            for algo in CONFIG["ALGO"]:
                präds = []

                if algo == 'RIPPER':
                    start_time = time.time()
                    try:
                        estimator = Ripper()
                        estimator.fit(x_train_bin, y_train, pos_value=CONFIG['POS_CLASS'])
                        end_time = time.time()
                        y_pred = estimator.predict(x_test_bin)
                        print('------------------------------------------------------') 
                        print('RIPPER QUANTILE')
                        #print('Accuracy:', accuracy_score(y_test, y_pred))
                        #print('Balanced accuracy:', balanced_accuracy_score(y_test, y_pred))
                        #print('Precision:', precision_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
                        #print('Recall:', recall_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
                        ripper_q_bacc = balanced_accuracy_score(y_test, y_pred, adjusted=True)
                        ripper_q_f2 = fbeta_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS'], beta= 2)
                        ripper_q_acc = accuracy_score(y_test, y_pred)
                        #print('ConfusionMatrix', confusion_matrix(y_test, y_pred))
                        #print('F-2', fbeta_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS'], beta= 2))
                        #print('Mathhews', matthews_corrcoef(y_test, y_pred))
                        #print('Rule count: ' + str(sum([len(rules) for rules in estimator.rule_map.values()])))
                        ripper_q_rl = str(sum([len(rules) for rules in estimator.rule_map.values()]))
                        if ripper_q_rl != 0:
                            
                            for i in range(len(estimator.rule_map[CONFIG['POS_CLASS']])):
                                präds.append(len(estimator.rule_map[CONFIG['POS_CLASS']][i]))
                            ripper_q_präd = sum(präds)
                            ripper_q_max_präd = max(präds)

                        else:
                            ripper_q_präd = 0
                            ripper_q_max_präd= 0
                        #print("Sum Prädikate:", sum(präds))
                        #print("MAX Prädikate:", max(präds))
                        
                        print('------------------------------------------------------')
                    except Exception:
                        pass
                if algo == 'BRCG':
                    start_time = time.time()
                    estimator = BooleanRuleCG()
                    with warnings.catch_warnings():
                        warnings.simplefilter("ignore")
                        estimator.fit(x_train_bin, y_train)
                        end_time = time.time()
                    y_pred = estimator.predict(x_test_bin)
                    print('------------------------------------------------------')  
                    print('BRCG QUANTILE')
                    #print('Accuracy:', accuracy_score(y_test, y_pred))
                    #print('Balanced accuracy:', balanced_accuracy_score(y_test, y_pred))
                    #print('Precision:', precision_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
                    #print('Recall:', recall_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
                    brcg_q_bacc = balanced_accuracy_score(y_test, y_pred, adjusted=True)
                    brcg_q_f2 = fbeta_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS'], beta= 2)
                    brcg_q_acc = accuracy_score(y_test, y_pred)
                    #print('ConfusionMatrix', confusion_matrix(y_test, y_pred))
                    #print('F-2', fbeta_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS'], beta= 2))
                    #print('Mathhews', matthews_corrcoef(y_test, y_pred))
                    model = estimator.explain()
                
                    #print('Number of rules:', len(model['rules']))
                    brcg_q_rl = len(model['rules'])
                    print('------------------------------------------------------')

                if algo == 'CORELS':
                    start_time = time.time()
                    estimator = CorelsClassifier(n_iter=10000, 
                        max_card=2, # feautres per statement
                        c = 0.0001 # Higher values penalise longer rulelists
                        )
                    estimator.fit(x_train_bin, y_train , prediction_name = CONFIG["TARGET_LABEL"])
                    end_time = time.time()
                    y_pred = estimator.predict(x_test_bin)
                    print('------------------------------------------------------') 
                    #print('CORELS QUANTILE')
                    #print('Accuracy:', accuracy_score(y_test, y_pred))
                    #print('Balanced accuracy:', balanced_accuracy_score(y_test, y_pred))
                    #print('Precision:', precision_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
                    #print('Recall:', recall_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
                    corels_q_bacc = balanced_accuracy_score(y_test, y_pred, adjusted=True)
                    corels_q_f2 = fbeta_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS'], beta= 2)
                    corels_q_acc = accuracy_score(y_test, y_pred)
                    #print('ConfusionMatrix', confusion_matrix(y_test, y_pred))
                    #print('F-2', fbeta_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS'], beta= 2))
                    #print('Mathhews', matthews_corrcoef(y_test, y_pred))
                    r_length = len(estimator.rl().rules)
                    corels_q_rl = len(estimator.rl().rules)
                    #print("Rule Length:", r_length)
                    print('------------------------------------------------------')   
                

        if i == "NATIVE":
            x_train_bin = x_train
            x_test_bin = x_test

            start_time = time.time()
            try:
                estimator = Ripper()
                estimator.fit(x_train_bin, y_train, pos_value=CONFIG['POS_CLASS'])
                end_time = time.time()
                y_pred = estimator.predict(x_test_bin)
                print('------------------------------------------------------') 
                print('RIPPER NATIVE')
                #print('Accuracy:', accuracy_score(y_test, y_pred))
                #print('Balanced accuracy:', balanced_accuracy_score(y_test, y_pred))
                #print('Precision:', precision_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
                #print('Recall:', recall_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
                ripper_n_bacc = balanced_accuracy_score(y_test, y_pred, adjusted=True)
                ripper_n_f2 = fbeta_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS'], beta= 2)
                ripper_n_acc = accuracy_score(y_test, y_pred)
                #print('ConfusionMatrix', confusion_matrix(y_test, y_pred))
                #print('F-2', fbeta_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS'], beta= 2))
                #print('Mathhews', matthews_corrcoef(y_test, y_pred))
                #print('Rule count: ' + str(sum([len(rules) for rules in estimator.rule_map.values()])))
                ripper_n_rl = str(sum([len(rules) for rules in estimator.rule_map.values()]))
                if ripper_n_rl != 0:
                
                    for i in range(len(estimator.rule_map[CONFIG['POS_CLASS']])):
                        präds.append(len(estimator.rule_map[CONFIG['POS_CLASS']][i]))
                    ripper_n_präd = sum(präds)
                    ripper_n_max_präd = max(präds)
    
                else:
                    ripper_n_präd = 0
                    ripper_n_max_präd= 0
            except Exception:
                pass

                #print("Sum Prädikate:", sum(präds))
                #print("MAX Prädikate:", max(präds))

    
    metric_dict.update({config:{"Config":config,"ripper_t_bacc":ripper_t_bacc, "ripper_t_f2": ripper_t_f2,"ripper_t_acc":ripper_t_acc,"ripper_t_rl":ripper_t_rl,"ripper_t_präd":ripper_t_präd,"ripper_t_max_präd":ripper_t_max_präd, 
                                                "brcg_t_bacc": brcg_t_bacc, "brcg_t_f2": brcg_t_f2,"brcg_t_acc":brcg_t_acc,"brcg_t_rl":brcg_t_rl,
                                                "corels_t_bacc":corels_t_bacc, "corels_t_f2": corels_t_f2,"corels_t_acc":corels_t_acc,"corels_t_rl":corels_t_rl,
                                                "ripper_q_bacc":ripper_q_bacc, "ripper_q_f2":ripper_q_f2,"ripper_q_acc":ripper_q_acc,"ripper_q_rl":ripper_q_rl,"ripper_q_präd":ripper_q_präd,"ripper_q_max_präd":ripper_q_max_präd, 
                                                "brcg_q_bacc":brcg_q_bacc,"brcg_q_f2":brcg_q_f2,"brcg_q_acc":brcg_q_acc,"brcg_q_rl":brcg_q_rl,
                                                "corels_q_bacc":corels_q_bacc,"corels_q_f2":corels_q_f2,"corels_q_acc": corels_q_bacc,"corels_q_rl":corels_q_rl,
                                                "ripper_n_bacc": ripper_n_bacc, "ripper_n_f2": ripper_n_f2,"ripper_n_acc": ripper_n_bacc, "ripper_n_rl":ripper_n_rl,"ripper_n_präd":ripper_n_präd,"ripper_n_max_präd":ripper_n_max_präd}})
    metric_list.append(metric_dict[config])
   

        
            

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Status Checking          1000 non-null   object 
 1   Duration in Month        1000 non-null   float64
 2   Credit History           1000 non-null   object 
 3   Purpose                  1000 non-null   object 
 4   Credit Amount            1000 non-null   float64
 5   Savings Account          1000 non-null   object 
 6   Employement since        1000 non-null   object 
 7   Installmentrate %        1000 non-null   float64
 8   StatusSex                1000 non-null   object 
 9   Otherdebtos              1000 non-null   object 
 10  PresentResidence         1000 non-null   float64
 11  Property                 1000 non-null   object 
 12  Age in years             1000 non-null   float64
 13  Otherinstallment Plans   1000 non-null   object 
 14  Housing                  

In [9]:
#metric_dict
metric_list

[{'Config': 'CONFIG-I1',
  'ripper_t_bacc': 0.21652032178347969,
  'ripper_t_f2': 0.6625441696113072,
  'ripper_t_acc': 0.5233333333333333,
  'ripper_t_rl': '17',
  'ripper_t_präd': 72,
  'ripper_t_max_präd': 7,
  'brcg_t_bacc': 0.15547610284452396,
  'brcg_t_f2': 0.36613272311212813,
  'brcg_t_acc': 0.6666666666666666,
  'brcg_t_rl': 3,
  'corels_t_bacc': 0.1527945738472054,
  'corels_t_f2': 0.25125628140703515,
  'corels_t_acc': 0.7166666666666667,
  'corels_t_rl': 3,
  'ripper_q_bacc': 0.2215679057784321,
  'ripper_q_f2': 0.3414634146341464,
  'ripper_q_acc': 0.73,
  'ripper_q_rl': '3',
  'ripper_q_präd': 11,
  'ripper_q_max_präd': 5,
  'brcg_q_bacc': 0.30984804669015187,
  'brcg_q_f2': 0.4875283446712018,
  'brcg_q_acc': 0.7266666666666667,
  'brcg_q_rl': 6,
  'corels_q_bacc': 0.14180556285819446,
  'corels_q_f2': 0.23929471032745592,
  'corels_q_acc': 0.14180556285819446,
  'corels_q_rl': 2,
  'ripper_n_bacc': 0.17903149482096836,
  'ripper_n_f2': 0.3056234718826406,
  'ripper_n_a

In [10]:
type(config_dict_imbalanced[config])

dict

In [11]:
df_list = []
csv_list = []
for i in range(len(Config_list)):
    
    if Config_list[i][1]['TYPE'] == "BINARY":
        if Config_list[i][1]["DATA_SET"] not in csv_list:
            temp_df = pd.read_csv(Config_list[i][1]["DATA_SET"])
            csv_list.append(Config_list[i][1]["DATA_SET"])
            #temp_df = temp_df.drop(columns=Config_list[i][1]['DROP'])
            temp_df= temp_df.rename(columns={temp_df[Config_list[i][1]['TARGET_LABEL']].name : 'TARGET_LABEL'})
            df_list.append(temp_df)
        
eval_df = pd.DataFrame(csv_list, columns=['Data_Set'])
eval_df["Target_1_pos"] = pd.Series('int32')
eval_df["Target_2_neg"] = pd.Series('int32')
eval_df["IB_Ratio"] = pd.Series()
eval_df["Num_Feautures"] = pd.Series('int32')
eval_df["Cat_Feautures"] = pd.Series('int32')
eval_df["Size_row"] = pd.Series('int32')
eval_df["Size_col"] = pd.Series('int32')

# Trees
eval_df["ripper_t_bacc"] = pd.Series()
eval_df["ripper_t_f2"] = pd.Series()
eval_df["ripper_t_acc"] = pd.Series()
eval_df["ripper_t_rl"] = pd.Series()
eval_df["ripper_t_präd"] = pd.Series()
eval_df["ripper_t_max_präd"] = pd.Series()

eval_df["brcg_t_bacc"] = pd.Series()
eval_df["brcg_t_f2"] = pd.Series()
eval_df["brcg_t_acc"] = pd.Series()
eval_df["brcg_t_rl"] = pd.Series()

eval_df["corels_t_bacc"] = pd.Series()
eval_df["corels_t_f2"] = pd.Series()
eval_df["corels_t_acc"] = pd.Series()
eval_df["corels_t_rl"] = pd.Series()

# Quantile
eval_df["ripper_q_bacc"] = pd.Series()
eval_df["ripper_q_f2"] = pd.Series()
eval_df["ripper_q_acc"] = pd.Series()
eval_df["ripper_q_rl"] = pd.Series()
eval_df["ripper_q_präd"] = pd.Series()
eval_df["ripper_q_max_präd"] = pd.Series()

eval_df["brcg_q_bacc"] = pd.Series()
eval_df["brcg_q_f2"] = pd.Series()
eval_df["brcg_q_acc"] = pd.Series()
eval_df["brcg_q_rl"] = pd.Series()

eval_df["corels_q_bacc"] = pd.Series()
eval_df["corels_q_f2"] = pd.Series()
eval_df["corels_q_acc"] = pd.Series()
eval_df["corels_q_rl"] = pd.Series()

# Ripper Native
eval_df["ripper_n_bacc"] = pd.Series()
eval_df["ripper_n_f2"] = pd.Series()
eval_df["ripper_n_acc"] = pd.Series()
eval_df["ripper_n_rl"] = pd.Series()
eval_df["ripper_n_präd"] = pd.Series()
eval_df["ripper_n_max_präd"] = pd.Series()

C:\Users\hornd\AppData\Local\R-MINI~1\envs\aix360i_v2_clone\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (2,8,11,16,17,18,19,20,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\hornd\AppData\Local\R-MINI~1\envs\aix360i_v2_clone\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  app.launch_new_instance()
C:\Users\hornd\AppData\Local\R-MINI~1\envs\aix360i_v2_clone\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
C:\Users\hornd\AppData\Local\R-MINI~1\envs\aix360i_v2_clone\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: The 

In [12]:
eval_df

,Data_Set,Target_1_pos,Target_2_neg,IB_Ratio,Num_Feautures,Cat_Feautures,Size_row,Size_col,ripper_t_bacc,ripper_t_f2,...,corels_q_bacc,corels_q_f2,corels_q_acc,corels_q_rl,ripper_n_bacc,ripper_n_f2,ripper_n_acc,ripper_n_rl,ripper_n_präd,ripper_n_max_präd
0,../data/german_credit_codiert.csv,int32,int32,NaN,int32,int32,int32,int32,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,../data/fraud_detection.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,../data/TaiwanCreditData.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,../data/miniloan-decisions-100K.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,../data/fraud_detection_duenn.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,../data/binary_bike_imbalanced.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,../data/fraud_oracle_clean.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,../data/miniloan_duenn.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
for frame in range(len(df_list)):
    
    metric = df_list[frame]["TARGET_LABEL"].value_counts()

    # Imbalanced Ratio = minor class
    if metric[0] > metric[1]:
        eval_df["Target_1_pos"].iloc[frame] = metric[1]
        eval_df["Target_2_neg"].iloc[frame] = metric[0]
    else:
        eval_df["Target_1_pos"].iloc[frame] = metric[0]
        eval_df["Target_2_neg"].iloc[frame] = metric[1]
        
    df_size_row = len(df_list[frame])
    df_size_col = len(df_list[frame].columns)
    df_num_feauture =  len(df_list[frame].select_dtypes(include=['int64', 'float64']).columns)
    df_cat_feauture =   len(df_list[frame].select_dtypes(include=['object']).columns)

    eval_df["IB_Ratio"].iloc[frame] = eval_df["Target_1_pos"].iloc[frame]/eval_df["Target_2_neg"].iloc[frame]
    eval_df["Size_row"].iloc[frame]   = df_size_row
    eval_df["Size_col"].iloc[frame]   = df_size_col
    
    eval_df["Num_Feautures"].iloc[frame] = df_num_feauture
    eval_df["Cat_Feautures"].iloc[frame] = df_cat_feauture

    # adding Metrics Trees
    eval_df["ripper_t_bacc"].iloc[frame] = metric_list[frame]["ripper_t_bacc"]
    eval_df["ripper_t_f2"].iloc[frame] = metric_list[frame]["ripper_t_f2"]
    eval_df["ripper_t_acc"].iloc[frame] = metric_list[frame]["ripper_t_acc"]
    eval_df["ripper_t_rl"].iloc[frame] = metric_list[frame]["ripper_t_rl"]
    eval_df["ripper_t_präd"].iloc[frame] = metric_list[frame]["ripper_t_präd"]
    eval_df["ripper_t_max_präd"].iloc[frame] = metric_list[frame]["ripper_t_max_präd"]
    


    eval_df["brcg_t_bacc"].iloc[frame] = metric_list[frame]["brcg_t_bacc"]
    eval_df["brcg_t_f2"].iloc[frame] = metric_list[frame]["brcg_t_f2"]
    eval_df["brcg_t_acc"].iloc[frame] = metric_list[frame]["brcg_t_acc"]
    eval_df["brcg_t_rl"].iloc[frame] = metric_list[frame]["brcg_t_rl"]

    eval_df["corels_t_bacc"].iloc[frame] = metric_list[frame]["corels_t_bacc"]
    eval_df["corels_t_f2"].iloc[frame] = metric_list[frame]["corels_t_f2"]
    eval_df["corels_t_acc"].iloc[frame] = metric_list[frame]["corels_t_acc"]
    eval_df["corels_t_rl"].iloc[frame] = metric_list[frame]["corels_t_rl"]

    # adding Metrics Qunatile
    eval_df["ripper_q_bacc"].iloc[frame] =  metric_list[frame]["ripper_q_bacc"]
    eval_df["ripper_q_f2"].iloc[frame] = metric_list[frame]["ripper_q_f2"]
    eval_df["ripper_q_acc"].iloc[frame] = metric_list[frame]["ripper_q_acc"]
    eval_df["ripper_q_rl"].iloc[frame] = metric_list[frame]["ripper_q_rl"]
    eval_df["ripper_q_präd"].iloc[frame] = metric_list[frame]["ripper_q_präd"]
    eval_df["ripper_q_max_präd"].iloc[frame] = metric_list[frame]["ripper_q_max_präd"]

    eval_df["brcg_q_bacc"].iloc[frame] = metric_list[frame]["brcg_q_bacc"]
    eval_df["brcg_q_f2"].iloc[frame] = metric_list[frame]["brcg_q_f2"]
    eval_df["brcg_q_acc"].iloc[frame] = metric_list[frame]["brcg_q_acc"]
    eval_df["brcg_q_rl"].iloc[frame] = metric_list[frame]["brcg_q_rl"]
    

    eval_df["corels_q_bacc"].iloc[frame] =  metric_list[frame]["corels_q_bacc"]
    eval_df["corels_q_f2"].iloc[frame] =  metric_list[frame]["corels_q_f2"]
    eval_df["corels_q_acc"].iloc[frame] =  metric_list[frame]["corels_q_acc"]
    eval_df["corels_q_rl"].iloc[frame] =  metric_list[frame]["corels_q_rl"]

    # adding Metrics Ripper Native
    eval_df["ripper_n_bacc"].iloc[frame] = metric_list[frame]["ripper_n_bacc"]
    eval_df["ripper_n_f2"].iloc[frame] = metric_list[frame]["ripper_n_f2"]
    eval_df["ripper_n_acc"].iloc[frame] = metric_list[frame]["ripper_n_acc"]
    eval_df["ripper_n_rl"].iloc[frame] = metric_list[frame]["ripper_n_rl"]
    eval_df["ripper_n_präd"].iloc[frame] = metric_list[frame]["ripper_n_präd"]
    eval_df["ripper_n_max_präd"].iloc[frame] = metric_list[frame]["ripper_n_max_präd"]


eval_df

C:\Users\hornd\AppData\Local\R-MINI~1\envs\aix360i_v2_clone\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,Data_Set,Target_1_pos,Target_2_neg,IB_Ratio,Num_Feautures,Cat_Feautures,Size_row,Size_col,ripper_t_bacc,ripper_t_f2,...,corels_q_bacc,corels_q_f2,corels_q_acc,corels_q_rl,ripper_n_bacc,ripper_n_f2,ripper_n_acc,ripper_n_rl,ripper_n_präd,ripper_n_max_präd
0,../data/german_credit_codiert.csv,300,700,0.428571,9,13,1000,22,0.216520,0.662544,...,0.141806,0.239295,0.141806,2.0,0.179031,0.305623,0.179031,3,7.0,3.0
1,../data/fraud_detection.csv,492,284315,0.001730,31,0,284807,31,0.787242,0.742574,...,0.000000,0.000000,0.000000,1.0,0.789251,0.815217,0.789251,4,9.0,3.0
2,../data/TaiwanCreditData.csv,6636,23364,0.284027,16,9,30000,25,0.000000,0.000000,...,0.264369,0.341795,0.264369,2.0,0.241422,0.321755,0.241422,3,10.0,5.0
3,../data/miniloan-decisions-100K.csv,21954,78046,0.281296,7,1,100000,9,0.000000,0.000000,...,0.795020,0.837547,0.795020,2.0,0.964981,0.972251,0.964981,14,43.0,4.0
4,../data/fraud_detection_duenn.csv,492,10000,0.049200,32,0,10492,32,0.000000,0.000000,...,0.439005,0.488372,0.439005,2.0,0.780359,0.812592,0.780359,3,8.0,3.0
5,../data/binary_bike_imbalanced.csv,298,8462,0.035216,12,4,8760,16,0.814985,0.477032,...,0.000000,0.000000,0.000000,1.0,0.372943,0.416667,0.372943,5,25.0,6.0
6,../data/fraud_oracle_clean.csv,853,13604,0.062702,1,33,30841,34,0.303484,0.313511,...,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0,25.0,6.0
7,../data/miniloan_duenn.csv,4000,78046,0.051252,7,0,82046,8,0.000000,0.000000,...,0.760468,0.815657,0.760468,2.0,0.967893,0.976155,0.967893,19,66.0,5.0


In [14]:
eval_df.to_csv("test4.csv",sep =",")

In [15]:
CONFIG['POS_CLASS']

False

In [16]:
y_test.value_counts()

True     3291
False    1209
Name: approval, dtype: int64

In [17]:
CONFIG['POS_CLASS']

False

In [18]:
df[CONFIG["TARGET_LABEL"]].value_counts()

True     11000
False     4000
Name: approval, dtype: int64